# <center> RNA-Seq Differential Gene Expression Notebook </center>

## <center>Preface</center>  
<p><center>This Notebook provides an interactive way to process, analyze, and view differential gene expressions from next-generation sequencing data.<br>The following steps serve as a guide, however, care should be taken when executing and interpreting results.</center><br>This notebook uses the following R packages: <li> - <a href="https://bioconductor.org/packages/release/bioc/html/Rsubread.html"><strong>Rsubread</strong></a> to produce a count matrix from provided .bam files using featureCounts (if matrix not already created). <br>- <a href="https://bioconductor.org/packages/release/bioc/html/DESeq2.html"><strong>DESeq2</strong></a> to process the count matrix and produce differential gene expression statistics. <br>- <a href="https://bioconductor.org/packages/release/bioc/html/clusterProfiler.html"><strong>clusterProfiler</strong></a> to annotate genes (GO, GSEA, KEGG) using database packages for mappings. </li><center><br><em>Note that despite following the framework below, it is up to <strong>you</strong> to determine which steps or values to change or ignore based on your knowledge of experimental details in obtaining data and your analysis goals.</em></center></p>

### sessionInfo()

R version 4.3.1 (2023-06-16 ucrt)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 19045)

Matrix products: default


locale:
[1] LC_COLLATE=English_Canada.utf8  LC_CTYPE=English_Canada.utf8    LC_MONETARY=English_Canada.utf8
[4] LC_NUMERIC=C                    LC_TIME=English_Canada.utf8    

time zone: America/Toronto
tzcode source: internal

attached base packages:
[1] grid      stats4    stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] optparse_1.7.5              qvalue_2.32.0               reticulate_1.39.0          
 [4] pheatmap_1.0.12             RColorBrewer_1.1-3          EnhancedVolcano_1.18.0     
 [7] ggrepel_0.9.6               ggplot2_3.5.1               clusterProfiler_4.8.3      
[10] edgeR_3.42.4                limma_3.56.2                DESeq2_1.40.2              
[13] SummarizedExperiment_1.30.2 Biobase_2.60.0              MatrixGenerics_1.12.3      
[16] matrixStats_1.4.1           GenomicRanges_1.52.0        GenomeInfoDb_1.36.4        
[19] IRanges_2.34.1              S4Vectors_0.38.1            BiocGenerics_0.46.0        
[22] Rsubread_2.14.2            

loaded via a namespace (and not attached):
 [1] DBI_1.2.3               bitops_1.0-8            gson_0.1.0              shadowtext_0.1.4       
 [5] gridExtra_2.3           rlang_1.1.4             magrittr_2.0.3          DOSE_3.26.2            
 [9] compiler_4.3.1          RSQLite_2.3.7           png_0.1-8               vctrs_0.6.5            
[13] reshape2_1.4.4          stringr_1.5.1           pkgconfig_2.0.3         crayon_1.5.3           
[17] fastmap_1.2.0           XVector_0.40.0          ggraph_2.2.1            utf8_1.2.4             
[21] HDO.db_0.99.1           enrichplot_1.20.3       purrr_1.0.2             bit_4.5.0              
[25] zlibbioc_1.46.0         cachem_1.1.0            aplot_0.2.3             jsonlite_1.8.9         
[29] blob_1.2.4              DelayedArray_0.26.6     BiocParallel_1.34.2     tweenr_2.0.3           
[33] parallel_4.3.1          R6_2.5.1                stringi_1.8.4           GOSemSim_2.26.1        
[37] Rcpp_1.0.13             downloader_0.4          Matrix_1.5-4.1          splines_4.3.1          
[41] igraph_2.0.3            tidyselect_1.2.1        rstudioapi_0.16.0       viridis_0.6.5          
[45] codetools_0.2-19        lattice_0.21-8          tibble_3.2.1            plyr_1.8.9             
[49] treeio_1.24.3           withr_3.0.1             KEGGREST_1.40.1         gridGraphics_0.5-1     
[53] getopt_1.20.4           scatterpie_0.2.4        polyclip_1.10-7         Biostrings_2.68.1      
[57] ggtree_3.8.2            pillar_1.9.0            ggfun_0.1.6             generics_0.1.3         
[61] RCurl_1.98-1.16         tidytree_0.4.6          munsell_0.5.1           scales_1.3.0           
[65] glue_1.7.0              lazyeval_0.2.2          tools_4.3.1             data.table_1.16.0      
[69] fgsea_1.26.0            locfit_1.5-9.10         fs_1.6.4                graphlayouts_1.2.0     
[73] fastmatch_1.1-4         tidygraph_1.3.1         cowplot_1.1.3           ape_5.8                
[77] tidyr_1.3.1             AnnotationDbi_1.62.2    colorspace_2.1-1        nlme_3.1-162           
[81] patchwork_1.3.0         GenomeInfoDbData_1.2.10 ggforce_0.4.2           cli_3.6.1              
[85] fansi_1.0.6             S4Arrays_1.0.4          viridisLite_0.4.2       dplyr_1.1.4            
[89] gtable_0.3.5            yulab.utils_0.1.7       digest_0.6.37           ggplotify_0.1.2        
[93] farver_2.1.2            memoise_2.0.1           lifecycle_1.0.4         httr_1.4.7             
[97] GO.db_3.17.0            bit64_4.5.2             MASS_7.3-60

### Required:
<br><strong>1) A <u>COUNT MATRIX FILE</u> produced by featureCounts and/or htseq-count which should be comma-separated (.csv), example formatted as follows:</strong><br>
<table style=’float:left;’>
    <tr>
        <th>GeneID</th>
        <th>BamSample 1</th>
        <th>BamSample 2</th>
        <th>BamSample 3</th>
        <th>BamSample N</th>
    </tr>
    <tr>
        <td>gene1</td>
        <td>123</td>
        <td>456</td>
        <td>78</td>
        <td>89</td>
    </tr>
    <tr>
        <td>gene2</td>
        <td>321</td>
        <td>654</td>
        <td>45</td>
        <td>12</td>
    </tr>
    <tr>
        <td>gene3</td>
        <td>456</td>
        <td>741</td>
        <td>0</td>
        <td>0</td>
    </tr>
    <tr>
        <td>geneN</td>
        <td>369</td>
        <td>987</td>
        <td>100</td>
        <td>200</td>
    </tr>
</table><br>
&emsp;<strong>There should be no duplicated gene IDs in the first column.</strong><br>
&emsp;<strong>---This file may be produced using this notebook below with the provided .bam files...or using <a href="https://github.com/earezza/Genomics-Pipelines/blob/main/DownstreamAnalysis/DifferentialAnalysis/get_rnaseq_counts.R">get_rnaseq_counts.R</a>---</strong>
<br><br>
<strong>2) A <u>SAMPLE INFO FILE</u> is also required and should be comma-separated (.csv), example formatted as follows:</strong><br>
<table style=’float:left;’>
    <tr>
        <th>Sample</th>
        <th>Condition</th>
    </tr>
    <tr>
        <td>BamSample 1</td>
        <td>control</td>
    </tr>
    <tr>
        <td>BamSample 2</td>
        <td>control</td>
    </tr>
    <tr>
        <td>BamSample 3</td>
        <td>treatment</td>
    </tr>
    <tr>
        <td>BamSample N</td>
        <td>treatment</td>
    </tr>
</table><br>
&emsp;<strong>Must contain a column with the header named "Condition" for grouping samples.</strong><br>
&emsp;<strong>There must be more than 1 sample for each condition.</strong><br>
&emsp;<strong>Row order does not matter.</strong><br>
&emsp;<strong>May contain multiple conditions. Pair-wise comparisons are performed for differential gene expressions between each pair combination.</strong>

## <center> ======= Load Packages ======= </center>

In [ ]:
# ======= Load Packages =======
suppressWarnings(suppressPackageStartupMessages({
  library(Rsubread)
  library(DESeq2)
  library(edgeR)
  library(clusterProfiler)
  library(ggplot2)
  library(EnhancedVolcano)
  #library(ensembldb)
  library(RColorBrewer)
  library(pheatmap)
  library(grid)
  library(reticulate)
  library(qvalue)
  library(optparse)
}))

## <center> =========== Define Functions ============ </center>

In [ ]:
# Load annotation database according to input params
load_annotation <- function(assembly, database){
  # ========= Get database references for annotations =========
  if (assembly == "mm10" | assembly == "mm9"){
    library(org.Mm.eg.db)
    annoDb <- "org.Mm.eg.db"
    keggOrg <- "mmu"
    if (database == "ucsc"){
      if (assembly == "mm10"){
        library(TxDb.Mmusculus.UCSC.mm10.knownGene)
        txdb <- TxDb.Mmusculus.UCSC.mm10.knownGene
      }else if (assembly == "mm9"){
        library(TxDb.Mmusculus.UCSC.mm9.knownGene)
        txdb <- TxDb.Mmusculus.UCSC.mm9.knownGene
      }
    }else if (database == "ensemble"){
      library(EnsDb.Mmusculus.v79)
      txdb <- EnsDb.Mmusculus.v79
      seqlevelsStyle(txdb) <- "UCSC" # format ensembl genes using UCSC style
    }
    else{
      stop("Invalid choice of annotation database")
    }
  } else if (assembly == "hg38" | assembly == "hg19"){
    library(org.Hs.eg.db)
    annoDb <- "org.Hs.eg.db"
    keggOrg <- "hsa"
    if (database == "ucsc"){
      if (assembly == "hg38"){
        library(TxDb.Hsapiens.UCSC.hg38.knownGene)
        txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene
      }else if (assembly == "hg19"){
        library(TxDb.Hsapiens.UCSC.hg19.knownGene)
        txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene
      }
    }else if (database == "ensemble"){
      library(EnsDb.Hsapiens.v86)
      txdb <- EnsDb.Hsapiens.v86
      seqlevelsStyle(txdb) <- "UCSC" # format ensembl genes using UCSC style
    } 
    else{
      stop("Invalid choice of annotation database")
    }
  } else if (assembly == "rn6"){
    library(TxDb.Rnorvegicus.UCSC.rn6.refGene)
    library(org.Rn.eg.db)
    annoDb <- "org.Rn.eg.db"
    keggOrg <- "rno"
    if (database == "ucsc"){
      txdb <- TxDb.Rnorvegicus.UCSC.rn6.refGene
    }else if (database == "ensemble"){
      library(EnsDb.Rnorvegicus.v79)
      txdb <- EnsDb.Rnorvegicus.v79
      seqlevelsStyle(txdb) <- "UCSC" # format ensembl genes using UCSC style
    } 
    else{
      stop("Invalid choice of annotation database")
    }
  } else{
    stop("Invalid choice of assembly")
  }
  
  anno_ref <- list("txdb"=txdb, "annoDb"=annoDb, "keggOrg"=keggOrg)
  
  return(anno_ref)
}

In [ ]:
# Dotplots for GO/KEGG pathway analysis
make_dotplot <- function(df, title="", ylabel="Description", colour="#56B1F7", n=15){
  df$ycolour <- "black"
  if ("ONTOLOGY" %in% colnames(df)){
    df$Description <- paste(df$ONTOLOGY, df$Description, sep=' - ')
    df$ycolour <- ifelse(grepl("BP -", df$Description), 'blue', df$ycolour)
    df$ycolour <- ifelse(grepl("CC -", df$Description), 'red', df$ycolour)
    df$ycolour <- ifelse(grepl("MF -", df$Description), 'darkgreen', df$ycolour)
  }
  df <- df[order(df$p.adjust, decreasing=FALSE),]
  
  # Re-format y-axis labels to not squish graph
  for (d in 1:length(df$Description)){
    i <- 1
    s <- df$Description[d]
    s <- str_remove(s, " - Mus musculus \\(house mouse\\)")
    df$Description[d] <- ""
    while (i < length(strsplit(s, ' ')[[1]]) + 1){
      df$Description[d] <- paste(df$Description[d], paste(strsplit(s, ' ')[[1]][i:(i+5)], collapse = ' '), sep='\n')
      i <- (i+6)
    }
    df$Description[d] <- gsub(" NA", "", df$Description[d])
    df$Description[d] <- substring(df$Description[d], 2, nchar(df$Description[d]))
  }
  # Plot
  plt <- ggplot() +
    geom_point(data=head(df, n=n),
               aes(x = -log(p.adjust), 
                   y = reorder(Description, -p.adjust), 
                   colour = Count, 
                   size = unname(unlist(sapply(GeneRatio, function(x) eval(parse(text=x)))))*100,
               ),
    ) + 
    theme_classic() +
    theme(axis.text.y = element_text(colour=rev(head(df$ycolour, n=n)))) +
    scale_color_gradient(low = "black", high = colour) +
    ggtitle(title)
  plt$labels$x <- "-log(p.adjust)"
  plt$labels$y <- ylabel
  plt$labels$size <- "GenePercentage"
  plt$labels$colour <- "GeneCount"
  return(plt)
}


---

In [ ]:
# Volcano plots for DEGs
make_volcanoplot <- function(res, condition1, condition2, lfc_threshold, padj_threshold, subtitle=''){
  df <- as.data.frame(res)
  
  keyvals <- ifelse(df$log2FoldChange <= (0-lfc_threshold), 'darkred', 'black')
  keyvals <- ifelse(df$log2FoldChange <= (0-lfc_threshold) & df$padj <= padj_threshold, 'red', keyvals)
  keyvals <- ifelse(df$log2FoldChange >= lfc_threshold, 'darkgreen', keyvals)
  keyvals <- ifelse(df$log2FoldChange >= lfc_threshold & df$padj <= padj_threshold, 'green', keyvals)
  
  keyvals[is.na(keyvals)] <- 'black'
  names(keyvals)[keyvals == 'darkgreen'] <- condition1
  names(keyvals)[keyvals == 'green'] <- paste(condition1, '(significant)', sep=' ')
  names(keyvals)[keyvals == 'black'] <- 'Not DEG'
  names(keyvals)[keyvals == 'darkred'] <- condition2
  names(keyvals)[keyvals == 'red'] <- paste(condition2, '(significant)', sep=' ')
  
  plt <- EnhancedVolcano(df, 
                         x = 'log2FoldChange', 
                         y = 'padj', 
                         xlab = "Log2FoldChange",
                         ylab = "-Log(adjusted p-value)",
                         caption = paste0("Total = ", nrow(df), " genes\n", 
                                          condition1, " = ", nrow(subset(df, log2FoldChange >= lfc_threshold)), ' DEGs\n',
                                          condition2, " = ", nrow(subset(df, log2FoldChange <= (0-lfc_threshold))), ' DEGS'
                         ),
                         colCustom = keyvals,
                         lab = rownames(df),
                         title = paste(condition1, ' vs ', condition2, sep=''),
                         subtitle = subtitle,
                         pCutoff = padj_threshold,
                         FCcutoff = lfc_threshold,
                         cutoffLineType = 'dashed',
                         pointSize = 2.0,
                         legendLabels=c('Not sig.','Log2FC','p-value','p-value & Log2FC'),
                         labSize = 3,
                         legendPosition = 'top',
                         legendLabSize = 6,
  )
  return(plt)
}

In [ ]:
make_heatmapplot <- function(res, condition1, condition2, n = 50, subtitle=''){
  df <- as.data.frame(head(res, n=n))
  df$Condition <- ifelse(df$log2FoldChange > 0, condition1, condition2)
  plt <- ggplot(df, aes(x = Condition, 
                        y = rownames(df), 
                        fill = log2FoldChange),
  ) +
    geom_tile() +
    ggtitle(paste("Top", n, "DEGs", sep=' '), subtitle=subtitle) +
    xlab("Condition") +
    ylab("Gene") 
  return(plt)
}

In [ ]:
# Heatmaps of top genes in GO/KEGG pathways
make_pheatmapplot <- function(anno, res, anno_type="GO", assembly='mm10', heat_colour="PiYG", num_terms=25, num_genes=50, lfc=0.6, dendro=TRUE, sort_genes=TRUE, title="", xlabel="Gene", ylabel="Term"){
  
  # colour should be "Reds", "Greens", "Blues", or "PiYG"
  if ("ONTOLOGY" %in% colnames(anno)){
    anno$Description <- paste(anno$ONTOLOGY, anno$Description, sep=' - ')
  }
  
  # Take top n terms (most significant, already sorted by padj)
  df <- head(anno[order(anno$p.adjust, decreasing=FALSE), ], n=num_terms)
  
  # Re-format y-axis labels to not squish graph
  for (d in 1:length(df$Description)){
    i <- 1
    s <- df$Description[d]
    if (length(strsplit(s, ' ')[[1]]) > 1){
      
      df$Description[d] <- ""
      while (i < length(strsplit(s, ' ')[[1]])){
        df$Description[d] <- paste(df$Description[d], paste(strsplit(s, ' ')[[1]][i:(i+5)], collapse = ' '), sep='\n')
        i <- (i+6)
      }
      df$Description[d] <- gsub(" NA", "", df$Description[d])
      df$Description[d] <- substring(df$Description[d], 2, nchar(df$Description[d]))
    }
  }
  
  # Create dataframe (matrix) of annotation terms vs genes with gene's associated log2FoldChange
  d <- data.frame()
  for (a in df$Description){
    gene_group <- strsplit(df[df$Description == a, ]$SYMBOL, '/')[[1]]
    # # For KEGG to convert EntrezID to gene Symbol
    # if (anno_type == "KEGG" | anno_type == "DAVID"){
    #   if (assembly == "hg19" | assembly == "hg38"){
    #     gene_group <- mapIds(org.Hs.eg.db, keys = gene_group, column = "SYMBOL", keytype = "ENTREZID")
    #   }else if (assembly == "mm9" | assembly == "mm10"){
    #     gene_group <- mapIds(org.Mm.eg.db, keys = gene_group, column = "SYMBOL", keytype = "ENTREZID")
    #   }else if (assembly == "rn6"){
    #     gene_group <- mapIds(org.Rn.eg.db, keys = gene_group, column = "SYMBOL", keytype = "ENTREZID")
    #   }
    # }
    d[gene_group, a] <- res[gene_group, ]$log2FoldChange
  }
  # Sort by genes instead of by term (i.e. number of times gene found in all top terms)
  if (sort_genes){
    d <- d[names(sort(rowSums(is.na(d)))), ]
    xlabel <- paste(xlabel, "(Most Frequent in Top Terms)")
  }
  # Set NA to 0
  d[is.na(d)] <- 0
  
  range_max <- round(max(apply(d, 2, max)))
  range_min <- round(min(apply(d, 2, min)))
  #breaks <- seq( -round(2^lfc), round(2^lfc), length.out = 101)
  breaks <- seq( -abs((max(range_min, -round(lfc)))), min(range_max, round(lfc)), length.out = 101)
  #breaks <- seq( -abs((max(range_min, -round(2^lfc)))), min(range_max, round(2^lfc)), length.out = 101)
  if (range_max == 0){
    color <- rev(colorRampPalette(brewer.pal(n = 11, name = heat_colour))(101))
  }else{
    color <- colorRampPalette(brewer.pal(n = 11, name = heat_colour))(101)
  }
  
  # Plot
  setHook("grid.newpage", function() pushViewport(viewport(x=0,y=0.05,width=0.95, height=0.95, name="vp", just=c("left","bottom"))), action="prepend")
  pheatmap(t(head(d, n=num_genes)), 
           border_color = "grey90",
           color = color, # "Reds, Greens, Blues, RdYlGn for DEGs
           fontsize_row = 6,
           fontsize_col = 6,
           na_col = "white",
           breaks = breaks,
           cluster_rows = dendro,
           cluster_cols = dendro,
           main = title,
  ) 
  setHook("grid.newpage", NULL, "replace")
  grid.text("log2FoldChange", x=0.95, y=0.875, gp=gpar(fontsize=8))
  grid.text(xlabel, y=0, gp=gpar(fontsize=14))
  grid.text(ylabel, x=1, y=0.35,  rot=270, gp=gpar(fontsize=14))
  plt <- grid.grab()
  
  return(plt)
}

In [ ]:
change_dirs <- function(res_dir, method, subfolder){
  if ((method == 'DESeq2')){
    method_dir <- paste(res_dir, 'DESeq2/', sep='')
    if (!file.exists(paste(method_dir, subfolder, '/', sep=''))) {
      dir.create(paste(method_dir, subfolder, '/', sep=''))
    }
  }else if ((method == 'edgeR')){
    method_dir <- paste(res_dir, 'edgeR/', sep='')
    if (!file.exists(paste(method_dir, subfolder, '/', sep=''))) {
      dir.create(paste(method_dir, subfolder, '/', sep=''))
    }
  }else{
    method_dir <- paste(res_dir, method, '/', sep='')
    if (!file.exists(paste(method_dir, subfolder, '/', sep=''))) {
      dir.create(paste(method_dir, subfolder, '/', sep=''))
    }
  }
  return(paste(method_dir, subfolder, '/', sep=''))
}

__________________________________________________________________________
### <center> Generate Count Matrix <br>(Skip if already provided)</center>

###  First, put all .bam (and .bai) files into a directory, e.g. "bams/", then run the following cells

In [ ]:
# Read in bam filenames from directory path
bam_files <- list.files(path="bams/", full.names=TRUE)[c(TRUE, FALSE)]

In [ ]:
# Count gene alignments (select appropriate assembly, mm10, hg38, etc...and check featureCount parameters) 
bamcounts <- featureCounts(bam_files, 
                           annot.inbuilt = "mm10",          # can be mm9, mm10, hg38, hg19
                           annot.ext = NULL,                # Path to external assembly annotation file if used (i.e. not using above inbuilt assembly)
                           isGTFAnnotationFile = FALSE,     # Set TRUE if using external assembly file and is .gtf formatted
                           countMultiMappingReads = FALSE,  # Set TRUE if want to include multi-mapped reads
                           ignoreDup = FALSE,           # Set TRUE if want to ignore duplicates (typically include duplicates for RNAseq data)
                           isPairedEnd = FALSE,         # Set TRUE if pair-end reads
                           strandSpecific = 0,          # 0 for not strand specific, 1 for counting forward reads only, 2 for counting reverse reads only
                           nthreads = 4,            # threads to speed up computation
                           verbose = TRUE)

In [ ]:
# Map gene NCBI Entrez IDs to Gene Symbols
library(org.Mm.eg.db)           # Load appropriate library for organism (mouse = org.Mm.eg.db, human = org.Hs.eg.db)
assembly_db <- org.Mm.eg.db     # Set to appropriate organism (mouse = org.Mm.eg.db, human = org.Hs.eg.db)
rownames(bamcounts$counts) <- mapIds(assembly_db, keys = rownames(bamcounts$counts), column = "SYMBOL", keytype = "ENTREZID")

In [ ]:
# Drop any unmapped genes
bamcounts$counts <- bamcounts$counts[!(is.na(rownames(bamcounts$counts))), ]

In [ ]:
# Write featureCounts output to file in current directory
for (n in names(bamcounts)){
  write.table(bamcounts[[n]], file=paste(getwd(), "/", n, ".csv", sep=""), sep=",", quote=F, col.names=NA)
}

#### Open counts.csv and modify header columns to names that will match your Sample column in your sampleinfo.csv file as mentioned above

### Check output files to ensure counts matrix was successfully created before proceeding.

---

## <center> ======= Define Input Parameters ======= </center>

In [ ]:
# ======= Get command-line optional arguments =======
option_list = list(
  make_option(c("-c", "--countsfile"), type="character", help="Count matrix file", metavar="character"),
  make_option(c("-s", "--sampleinfo"), type="character", help="File containing samples and their experimental conditions info (row names must match matrix file column names)", metavar="character"),
  make_option(c("-a", "--assembly"), type="character", default="mm10", help="Assembly to annotate genes/peaks (e.g. hg19, hg38, mm9, mm10, rn6)", metavar="character"),
  make_option(c("-d", "--database"), type="character", default="ucsc", help="Database reference for peaks gene annotations, ucsc (default) or ensembl", metavar="character"),
  make_option(c("-r", "--result_dir"), type="character", default="DEG_Analysis/", help="Directory name for saving output results", metavar="character"),
  make_option(c("-m", "--method"), type="character", default="all", help="Method to analyze data, ('deseq2', 'edger', 'limma', default: 'all')", metavar="character"), 
  make_option(c("-f", "--filter"), action="store_true", type="logical", default=FALSE, help="Flag to filter read counts (removes genes < min_count from raw matrix, then removes genes < min_baseMean after normalizing", metavar="logical"),
  make_option(c("--min_count"), type="integer", default=1, help="Minimum counts to include if filtering", metavar="integer"),
  make_option(c("--min_basemean"), type="double", default=10, help="Minimum baseMean of normalized counts to include if filtering", metavar="integer"),
  make_option(c("--lfc"), type="double", default=0.585, help="Magnitude of log2foldchange to define significant up/down regulation of genes", metavar="double"),
  make_option(c("--pvalue"), type="double", default=0.05, help="Significance threshold for DEGs (pvalue instead of p.adjusted for case where small sample set results in p.adjust=NA)", metavar="double"),
  make_option(c("--david_user"), type="character", default="earezza@ohri.ca", help="User email for DAVID web tools (must be registered, https://david.ncifcrf.gov/content.jsp?file=DAVID_WebService.html)", metavar="character"),
  make_option(c("--minGSSize"), type="integer", default=10, help="minimal size of genes annotated for testing", metavar="integer"),
  make_option(c("--maxGSSize"), type="integer", default=500, help="maximal size of genes annotated for testing", metavar="integer")
);
opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);

cat("Command-line options:\n")
for (i in which(names(opt) != "help")) {
  cat(names(opt)[i], '=', paste(opt)[i], "\n")
}
cat("log2FC of", opt$lfc, "equates to FC of", 2^opt$lfc)

#### Manually set parameters (if different from defaults)

In [ ]:
opt$countsfile <- ""
opt$sampleinfo <- ""
opt$assembly <- "mm10"
opt$filter <- TRUE

#### View current parameters

In [ ]:
# View variables
for (i in which(names(opt) != "help")) {
  cat(names(opt)[i], '=', paste(opt)[i], "\n")
}

# <center> ========= START DIFFERENTIAL EXPRESSION ANALYSIS ========= </center>
In this analysis, three commonly used tools (DESeq2, edgeR, and Limma) are implemented to perform the same task with similar steps grouped sequentially after each other.  As with many bioinformatics analyses, different tools may produce different results; however, if multiple tools produce confluent results then you can be more confident in your findings.  
With that in mind, feel free to run one, two, or all three of the tools on your data to achieve your investigation goals.  

#### Load Input Files and Annotation Info

In [ ]:
anno_ref <- load_annotation(opt$assembly, opt$database)
count_mtx <- as.matrix(read.csv(opt$countsfile, sep=",", row.names=1, check.names=FALSE))
sampleinfo <- read.csv(opt$sampleinfo, row.names=1)
# change to R syntactically correct names
sampleinfo$Condition <- gsub('-', '_', sampleinfo$Condition)
# Use only counts for samples listed in sample info file
count_mtx <- count_mtx[, rownames(sampleinfo)]
if (!all(colnames(count_mtx) == rownames(sampleinfo))){
  cat("Check that column names in counts file matches row names in sample info file.\n")
  q()
}

View some counts to verify input

In [ ]:
head(as.data.frame(count_mtx))

View sample info

In [ ]:
sampleinfo

Create output directory if doesn't exist

In [ ]:
# Create output file directory and set as working directory
if (!file.exists(opt$result_dir)) {
  dir.create(opt$result_dir)
}
cat("Output files will be in", opt$result_dir, "\n")

## <center> =========== Run Analysis ============ </center>

### PCA plot of all samples (DESeq2)

In [ ]:
# DESeq2
if (opt$method == 'deseq2' | opt$method == 'all') {
  output_prefix <- change_dirs(opt$result_dir, 'DESeq2', '')
  dds <- DESeqDataSetFromMatrix(countData = count_mtx,
                                colData = sampleinfo,
                                design = ~ Condition)
  cat("\nGenes and samples raw:", dim(dds), "\n")
  cat("DESEq2 genes count sums over all samples:\n")
  show(summary(rowSums(counts(dds))))
  show(quantile(rowSums(counts(dds)), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95)))
  if (opt$filter) {
    cat("Filtering counts...\n")
    # Remove 0 counts (sum of all samples' counts) to avoid 0-bias and disregard irrelevant genes
    keep_dds <- rowSums(counts(dds)) >= opt$min_count
    dds <- dds[keep_dds, ]
    cat("\nGenes and samples after removing genes with <", opt$min_count, "total counts:", dim(dds), "\n")
    show(summary(rowSums(counts(dds))))
    show(quantile(rowSums(counts(dds)), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95)))
    
    # Remove bottom 10 percentile counts from remaining
    low_counts <- quantile(rowSums(counts(dds)), probs = c(0.10))
    keep_dds <- rowSums(counts(dds)) > low_counts
    dds <- dds[keep_dds, ]
    cat("\nGenes and samples after removing bottom 10% genes with <", low_counts, "total counts:", dim(dds), "\n")
    show(summary(rowSums(counts(dds))))
    show(quantile(rowSums(counts(dds)), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95)))
  }
  dds <- DESeq(dds)
  
  vsd <- vst(dds, blind=FALSE)
  plt <- plotPCA(vsd, intgroup=c("Condition")) + 
    geom_text(aes(label = rownames(vsd@colData)), nudge_y = 0.2, size = 3)
  invisible(capture.output(ggsave(paste(output_prefix, "PCAplot.png", sep=''), plot=plt)))
}

### PCA plot of all samples (edgeR)

In [ ]:
# edgeR
if (opt$method == 'edger' | opt$method == 'all') {
  output_prefix <- change_dirs(opt$result_dir, 'edgeR', '')
  edge <- DGEList(counts=count_mtx, 
                  group=sampleinfo$Condition, 
                  remove.zeros=FALSE)
  cat("\nGenes and samples raw:", dim(edge), "\n")
  cat("EdgeR genes count sums over all samples:\n")
  show(summary(rowSums(edge$counts)))
  show(quantile(rowSums(edge$counts), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95)))
  if (opt$filter) {
    cat("Filtering counts...\n")
    # Remove 0 counts (sum of all samples' counts) to avoid 0-bias and disregard irrelevant genes
    keep_edge <- rowSums(edge$counts) >= opt$min_count
    edge <- edge[keep_edge, ]
    cat("\nGenes and samples after removing genes with <", opt$min_count, "total counts:", dim(edge), "\n")
    show(summary(rowSums(edge$counts)))
    show(quantile(rowSums(edge$counts), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95)))
    
    # Remove bottom 10 percentile counts from remaining
    low_counts <- quantile(rowSums(edge$counts), probs = c(0.10))
    keep_edge <- rowSums(edge$counts) > low_counts
    edge <- edge[keep_edge, ]
    cat("\nGenes and samples after removing bottom 10% genes with <", low_counts, "total counts:", dim(edge), "\n")
    show(summary(rowSums(edge$counts)))
    show((quantile(rowSums(edge$counts), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95))))
  }
  edge <- calcNormFactors(edge, method="TMM")
  normalized_count_mtx_edge <- cpm(edge, normalized.lib.sizes=TRUE) 
  edge <- DGEList(counts = normalized_count_mtx_edge, group=sampleinfo$Condition, remove.zeros=FALSE, norm.factors = edge$samples$norm.factors)
  
  # Plot PCA
  png(paste(output_prefix, 'MDSplot.png', sep=''))
  plotMDS(edge, col=as.numeric(edge$samples$group), plot=TRUE)
  invisible(capture.output(dev.off()))
}

### PCA plot of all samples (Limma)

In [ ]:
# Limma
if (opt$method == 'limma' | opt$method == 'all') {
  output_prefix <- change_dirs(opt$result_dir, 'Limma', '')
  lim <- DGEList(count_mtx)
  lim$samples$group <- sampleinfo$Group[which(rownames(sampleinfo) == rownames(lim$samples))]
  lim <- calcNormFactors(lim)
  cat("\nGenes and samples raw:", dim(lim), "\n")
  cat("Limma genes count sums over all samples:\n")
  show(summary(rowSums(lim$counts)))
  show(quantile(rowSums(lim$counts), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95)))
  
  if (opt$filter) {
    cat("Filtering counts...\n")
    # Remove 0 counts (sum of all samples' counts) to avoid 0-bias and disregard irrelevant genes
    keep_lim <- rowSums(lim$counts) >= opt$min_count
    lim <- lim[keep_lim,]
    cat("\nGenes and samples after removing genes with <", opt$min_count, "total counts:", dim(lim), "\n")
    show(summary(rowSums(lim$counts)))
    show(quantile(rowSums(lim$counts), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95)))
    
    # Remove bottom 10 percentile counts from remaining
    low_counts <- quantile(rowSums(lim$counts), probs = c(0.10))
    keep_lim <- rowSums(lim$counts) > low_counts
    lim <- lim[keep_lim, ]
    cat("\nGenes and samples after removing bottom 10% genes with <", low_counts, "total counts:", dim(lim), "\n")
    show(summary(rowSums(lim$counts)))
    show(quantile(rowSums(lim$counts), probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95)))
  }
  # Plot PCA
  png(paste(output_prefix, 'MDSplot.png', sep=''))
  plotMDS(lim, col=as.numeric(lim$samples$group), plot=TRUE)
  invisible(capture.output(dev.off()))
}

### Obtain pair-wise combinations of sample conditions to compare

In [ ]:
combs <- as.data.frame(combn(unique(sampleinfo$Condition), 2))
combs

In [ ]:
# Iterate through condition combinations to compare DEG
for (c in colnames(combs)){
  
  comparison <- paste(combs[[c]][1], combs[[c]][2], sep="_vs_")
  cat("\n==========", comparison, "==========\n")
  # Isolate matrix for relevant samples
  samples1 <- rownames(sampleinfo[sampleinfo$Condition == combs[[c]][1], ])
  samples2 <- rownames(sampleinfo[sampleinfo$Condition == combs[[c]][2], ])
  mtx <- count_mtx[, c(samples1, samples2)]
  mtx_info <- sampleinfo[c(samples1, samples2), ]
  mtx_info$Condition <- factor(mtx_info$Condition)
  
  # Check that matrix columns are same order as sample info
  if (!all(rownames(mtx_info) == colnames(mtx))) {
    break
  }
  
  # Create DESeqDataSet object from input
  dds <- DESeqDataSetFromMatrix(countData = mtx,
                                colData = mtx_info,
                                design = ~ Condition)
  
  # Create directory for output files of current comparison
  if (!file.exists(comparison)) {
    dir.create(comparison)
  }
  
  # Set output filenames prefix
  output_prefix <- paste(getwd(), "/", comparison, "/", sep="")
  
  cat("Genes and samples:\n", dim(dds), "\n")
  
  # =========== Filter Count Matrix ============
  if (opt$filter) {
    
    # Get percentile ranges of counts
    cat("Percentiles of genes count sums over all samples:\n")
    cat(quantile(rowSums(counts(dds)), probs = c(0, 0.1, 0.25, 0.5, 0.75, 0.9, 1)), "\n")
    
    # Remove 0 counts to avoid 0-bias and disregard irrelevant genes
    keep <- rowSums(counts(dds)) >= opt$min_count
    dds <- dds[keep,]
    cat("Percentiles after removing genes with sum(counts) <", opt$min_count, "\n")
    quantile(rowSums(counts(dds)), probs = c(0, 0.1, 0.25, 0.5, 0.75, 0.9, 1))
    # Get relevant percentiles
    quant <- quantile(rowSums(counts(dds)), probs = c(0, 0.1, 0.25, 0.5, 0.75, 0.9, 1))
    # Remove genes with <= 0.1 percentile counts
    cat("Removing genes with sum(counts) <=", quant[[2]])
    keep <- rowSums(counts(dds)) > quant[[2]]
    dds <- dds[keep,]
    # Keep genes only where there are at least 3 samples having at least that count
    cat("Removing genes with sum(counts) <=", quant[[2]], "in at least", round(length(colnames(dds))/4), "samples\n")
    keep <- rowSums(counts(dds) > quant[[2]]) >= round(length(colnames(dds))/4)
    dds <- dds[keep,]
    
    cat("Genes and samples after filtering raw counts:\n", dim(dds), "\n")
  }
  
  # ========== Normalize and Compute DEG Stats =============
  # Create DESeq object
  dds <- DESeq(dds)
  
  # Write filtered count matrix to file
  write.table(counts(dds), file=paste(output_prefix, "count_mtx_filtered.csv", sep=""), sep=",", quote=F, col.names=NA)
  
  # Write normalized values to file
  normalized_count_mtx <- counts(dds, normalized=TRUE)
  write.table(normalized_count_mtx, file=paste(output_prefix, "count_mtx_normalized.csv", sep=""), sep=",", quote=F, col.names=NA)
  
  # Obtain DEG results and output to file
  res <- results(dds, contrast=c("Condition", combs[[c]][1], combs[[c]][2]))
  res[['FoldChange']] <- 2^abs(res[['log2FoldChange']])*(res[['log2FoldChange']]/abs(res[['log2FoldChange']]))
  write.table(res[order(res$log2FoldChange, decreasing=TRUE), ], file=paste(output_prefix, "DESeq2_FullResult_", comparison, ".csv", sep=""), sep=",", quote=F, col.names=NA)
  
  #resLFC <- lfcShrink(dds, coef=resultsNames(dds)[-1], type="apeglm")
  
  # Filter out normalized low-expressed genes (bottom 10th percentile) by baseMean
  cat("Genes and samples before filtering normalized genes:\n", dim(res), "\n")
  quant <- quantile(res$baseMean, probs = c(0, 0.1, 0.25, 0.5, 0.75, 0.9, 1))
  cat("Removing genes with baseMean expression in bottom 10th percentile <=", quant[[2]], "\n")
  res <- res[res$baseMean >= quant[[2]], ]
  cat("Genes and samples after filtering normalized genes:\n", dim(res), "\n")
  
  # Generate plots
  png(paste(output_prefix, 'MAplot.png', sep=''))
  plotMA(res)
  abline(h=c((0-opt$lfc), opt$lfc), col="red", lwd=2)
  invisible(capture.output(dev.off()))
  
  #png(paste(output_prefix, 'resLFC_MAplot.png', sep=''))
  #plotMA(resLFC, ylim=c(-2,2))
  #invisible(capture.output(dev.off()))
  
  #png(paste(output_prefix, 'res_genecount_minpadj.png', sep=''))
  #plotCounts(dds, gene=which.min(res$padj), intgroup="Condition")
  #invisible(capture.output(dev.off()))
  
  vsd <- vst(dds, blind=FALSE)
  png(paste(output_prefix, 'PCAplot.png', sep=''))
  plotPCA(vsd, intgroup=c("Condition"))
  invisible(capture.output(dev.off()))
  
  # ========== Organize DEGs ==============
  # Remove any NAs
  #res <- res[rowSums(is.na(res)) == 0, ]
  
  # Get significantly Upregulated (log2FoldChange > 0)
  res_up <- res[res$log2FoldChange >= opt$lfc ,]
  res_up_sorted <- res_up[order(res_up$log2FoldChange, decreasing=TRUE),]
  write.table(res_up_sorted, file=paste(output_prefix, "DESeq2_Result_", combs[[c]][1], ".csv", sep=""), sep=",", quote=F, col.names=NA)
  
  # Get significantly Downregulated (log2FoldChange < 0)
  res_down <- res[res$log2FoldChange <= (0 - opt$lfc) ,]
  res_down_sorted <- res_down[order(res_down$log2FoldChange, decreasing=TRUE),]
  write.table(res_down[order(res_down$log2FoldChange, decreasing=FALSE),], file=paste(output_prefix, "DESeq2_Result_", combs[[c]][2], ".csv", sep=""), sep=",", quote=F, col.names=NA)
  
  # Get all significantly changed genes
  res_changed <- res[(res$log2FoldChange >= opt$lfc | res$log2FoldChange <= (0 - opt$lfc)),]
  res_changed_sorted <- res_changed[order(abs(res_changed$log2FoldChange), decreasing=TRUE),]
  write.table(res_changed_sorted, file=paste(output_prefix, "DESeq2_Result_DEGs", ".csv", sep=""), sep=",", quote=F, col.names=NA)
  
  genes <- list()
  genes[[combs[[c]][1]]] <- rownames(res_up_sorted)
  genes[[combs[[c]][2]]] <- rownames(res_down_sorted)
  genes[["DEGs"]] <- rownames(res_changed_sorted)
  
  genes_entrez <- list()
  for (n in names(genes)){
    if (tolower(opt$organism) == "human"){
      genes_entrez[[n]] <- mapIds(org.Hs.eg.db, keys = genes[[n]], column = "ENTREZID", keytype = "SYMBOL")
    }else if (tolower(opt$organism) == "mouse"){
      genes_entrez[[n]] <- mapIds(org.Mm.eg.db, keys = genes[[n]], column = "ENTREZID", keytype = "SYMBOL")
    }
  }
  
  # ============= GO and KEGG Annotations ==============
  for (n in names(genes)){
    
    # GO Annotation
    tryCatch(
      {
        compGO <- compareCluster(geneCluster=genes[n],
                                 OrgDb=annoDb,
                                 fun="enrichGO",
                                 keyType="SYMBOL", #ENSEMBL
                                 pvalueCutoff=0.05,
                                 pAdjustMethod="BH",
                                 readable=TRUE
        ) # Check https://www.genome.jp/kegg/catalog/org_list.html for organism hsa=human mmu=mouse
        if (!is.null(compGO)){
          plt <- dotplot(compGO, showCategory = 8, title = paste("GO -", n, sep=""))
          invisible(capture.output(ggsave(filename=paste(output_prefix, 'GO_annotation_', n, '.png', sep=''), plot=plt, dpi=320)))
          # Write annotations to csv
          write.table(as.data.frame(compGO), file=paste(output_prefix, 'GO_annotation_', n, '.tsv', sep=''), sep="\t", quote=F, row.names=F, col.names=T)
        }
      },error = function(e)
      {
        message(e)
      }
    )
    
    # KEGG Annotation
    tryCatch(
      {
        # fun is "groupGO", "enrichGO", "enrichKEGG", "enrichDO" or "enrichPathway" 
        compKEGG <- compareCluster(geneCluster=genes_entrez[n],
                                   fun="enrichKEGG",
                                   pvalueCutoff=0.05,
                                   pAdjustMethod="BH",
                                   organism=keggOrg
        ) # Check https://www.genome.jp/kegg/catalog/org_list.html for organism hsa=human mmu=mouse
        if (!is.null(compKEGG)){
          plt <- dotplot(compKEGG, showCategory = 8, title = paste("KEGG -", n, sep=""))
          invisible(capture.output(ggsave(filename=paste(output_prefix, 'KEGG_annotation_', n, '.png', sep=''), plot=plt, dpi=320)))
          
          # Write annotations to csv
          write.table(as.data.frame(compKEGG), file=paste(output_prefix, 'KEGG_annotation_', n, '.tsv', sep=''), sep="\t", quote=F, row.names=F, col.names=T)
        }
      },error = function(e)
      {
        message(e)
      }
    ) 
    
    # GSEA
    tryCatch(
      {
        original_gene_list <- res[genes[[n]],]$log2FoldChange
        names(original_gene_list) <- rownames(res[genes[[n]],])
        gene_list <- na.omit(original_gene_list)
        
        gsea <- gseGO(geneList=gene_list, 
                      ont ="ALL", 
                      keyType = "SYMBOL", 
                      nPerm = 10000, 
                      minGSSize = 3, 
                      maxGSSize = 800, 
                      pvalueCutoff = 0.05, 
                      verbose = TRUE, 
                      OrgDb = annoDb, 
                      pAdjustMethod = "BH"
        )
        if (!is.null(gsea)){
          plt <- dotplot(gsea, showCategory = 8, title = paste("GSEA -", n, sep=""))
          invisible(capture.output(ggsave(filename=paste(output_prefix, 'GSEA_annotation_', n, '.png', sep=''), plot=plt, dpi=320)))
          
          # Write annotations to csv
          write.table(as.data.frame(compKEGG), file=paste(output_prefix, 'GSEA_annotation_', n, '.tsv', sep=''), sep="\t", quote=F, row.names=F, col.names=T)
        }
      },error = function(e)
      {
        message(e)
      }
    ) 
  }
  
}

## <center> =========== END ============ </center>